# **Bayesian Networks**

# Create data

In [25]:
import numpy as np
import pandas as pd
data = pd.DataFrame(data={'fruit': ["banana", "apple", "banana", "apple", "banana","apple", "banana", 
                                    "apple", "apple", "apple", "banana", "banana", "apple", "banana",], 
                          'tasty': ["yes", "no", "yes", "yes", "yes", "yes", "yes", 
                                    "yes", "yes", "yes", "yes", "no", "no", "no"], 
                          'size': ["large", "large", "large", "small", "large", "large", "large",
                                    "small", "large", "large", "large", "large", "small", "small"]})
print(data)

     fruit tasty   size
0   banana   yes  large
1    apple    no  large
2   banana   yes  large
3    apple   yes  small
4   banana   yes  large
5    apple   yes  large
6   banana   yes  large
7    apple   yes  small
8    apple   yes  large
9    apple   yes  large
10  banana   yes  large
11  banana    no  large
12   apple    no  small
13  banana    no  small


In [26]:
from pgmpy.models import BayesianNetwork
model = BayesianNetwork([('fruit', 'tasty'), ('size', 'tasty')])  # fruit -> tasty <- size

## State counts

Parameter learning is the task to estimate the values of the conditional probability distributions (CPDs), for the variables fruit, size, and tasty.

To make sense of the given data, we can start by counting how often each state of the variable occurs. If the variable is dependent on parents, the counts are done conditionally on the parents states, i.e. for seperately for each parent configuration:

In [21]:
from pgmpy.estimators import ParameterEstimator

pe = ParameterEstimator(model, data)
print("\n", pe.state_counts('fruit'))  # unconditional
print("\n", pe.state_counts('tasty'))  # conditional on fruit and size


         fruit
apple       7
banana      7

 fruit apple       banana      
size  large small  large small
tasty                         
no      1.0   1.0    1.0   1.0
yes     3.0   2.0    5.0   0.0


We can see, for example, that as many apples as bananas were observed and that 5 large bananas were tasty, while only 1 was not.



## Bayesian Parameter Estimation

In [22]:
from pgmpy.estimators import BayesianEstimator

model.fit(data, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in model.get_cpds():
    print(cpd)

+---------------+-----+
| fruit(apple)  | 0.5 |
+---------------+-----+
| fruit(banana) | 0.5 |
+---------------+-----+
+------------+---------------------+-----+--------------------+
| fruit      | fruit(apple)        | ... | fruit(banana)      |
+------------+---------------------+-----+--------------------+
| size       | size(large)         | ... | size(small)        |
+------------+---------------------+-----+--------------------+
| tasty(no)  | 0.30952380952380953 | ... | 0.7222222222222222 |
+------------+---------------------+-----+--------------------+
| tasty(yes) | 0.6904761904761905  | ... | 0.2777777777777778 |
+------------+---------------------+-----+--------------------+
+-------------+----------+
| size(large) | 0.657895 |
+-------------+----------+
| size(small) | 0.342105 |
+-------------+----------+


The estimated values in the CPDs are now more conservative. In particular, the estimate for a small banana being not tasty is now around 0.64 rather than 1.0. Setting equivalent_sample_size to 10 means that for each parent configuration, we add the equivalent of 10 uniform samples (here: +5 small bananas that are tasty and +5 that aren't).

BayesianEstimator, too, can be used via the fit()-method. Full example:

In [27]:
model.fit(data, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in model.get_cpds():
    print(cpd)


+---------------+-----+
| fruit(apple)  | 0.5 |
+---------------+-----+
| fruit(banana) | 0.5 |
+---------------+-----+
+------------+---------------------+-----+--------------------+
| fruit      | fruit(apple)        | ... | fruit(banana)      |
+------------+---------------------+-----+--------------------+
| size       | size(large)         | ... | size(small)        |
+------------+---------------------+-----+--------------------+
| tasty(no)  | 0.30952380952380953 | ... | 0.7222222222222222 |
+------------+---------------------+-----+--------------------+
| tasty(yes) | 0.6904761904761905  | ... | 0.2777777777777778 |
+------------+---------------------+-----+--------------------+
+-------------+----------+
| size(large) | 0.657895 |
+-------------+----------+
| size(small) | 0.342105 |
+-------------+----------+


## **Inference in Bayesian Models**
Let's take an example for inference using Variable Elimination in pgmpy:


In [28]:
from pgmpy.inference import VariableElimination
infer = VariableElimination(model)
print(infer.query(['tasty']))
print(infer.query(['tasty'], evidence={'fruit': 'banana', 'size': 'large'}))


+------------+--------------+
| tasty      |   phi(tasty) |
+============+==============+
| tasty(no)  |       0.3645 |
+------------+--------------+
| tasty(yes) |       0.6355 |
+------------+--------------+
+------------+--------------+
| tasty      |   phi(tasty) |
+============+==============+
| tasty(no)  |       0.2241 |
+------------+--------------+
| tasty(yes) |       0.7759 |
+------------+--------------+


# **Predicting values from new data points**
In pgmpy this is known as MAP query. Here's an example:

In [18]:
print(infer.map_query(['tasty'], evidence={'fruit': 'banana', 'size': 'large'}))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

{'tasty': 'yes'}
